In [9]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch import Tensor

In [10]:
url='https://drive.google.com/uc?id=1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec'
data = pd.read_csv(url)

In [11]:
data

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                       0.351   31        0  
2                       0.672   32        1  
3                       0.167   21        0  
4                       2.288   33        1  
..                        ...  ...      ...  
763                     0.171   63        0  
764                     0.340   27        0  
765                     0.245   30        0  
766                     0.349   47        1  
767                     0.315   23        0  

[768 rows x 9 columns]

In [12]:
X = data.drop(columns=['Outcome'])
y = data['Outcome']

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [14]:
import torch.nn as nn
import torch.optim as optim

class NeuralNet3Layers(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet3Layers, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

class NeuralNet4Layers(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet4Layers, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))
        return x

In [27]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer, num_epochs=25):
    epoch_progress_bar = tqdm(range(num_epochs), desc='Training', unit='epoch')
    for epoch in epoch_progress_bar:
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        epoch_progress_bar.set_postfix(avg_loss=avg_loss)

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predicted = (outputs > 0.5)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

input_size = X_train.shape[1]
num_epochs = 100
learning_rate = 0.001

model3 = NeuralNet3Layers(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

print("Training 3-layer model...")
train_model(model3, train_loader, criterion, optimizer, num_epochs)
accuracy3 = evaluate_model(model3, test_loader)
print(f'Accuracy of 3-layer model: {accuracy3:.4f}')

model4 = NeuralNet4Layers(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model4.parameters(), lr=learning_rate)

print("Training 4-layer model...")
train_model(model4, train_loader, criterion, optimizer, num_epochs)
accuracy4 = evaluate_model(model4, test_loader)
print(f'\nAccuracy of 4-layer model: {accuracy4:.4f}')


Training 3-layer model...


Training: 100%|██████████| 100/100 [00:06<00:00, 14.94epoch/s, avg_loss=0.0156]


Accuracy of 3-layer model: 0.7532
Training 4-layer model...


Training: 100%|██████████| 100/100 [00:05<00:00, 17.79epoch/s, avg_loss=0.00213]


Accuracy of 4-layer model: 0.7468
